# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [91]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from datetime import datetime

# Import API key
from api_keys import geoapify_key

In [92]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tosontsengel,48.7567,98.2839,-22.06,99,97,0.90,MN,1701111187
1,1,port-aux-francais,-49.3500,70.2167,4.40,76,82,8.85,TF,1701111187
2,2,cadale,2.7603,46.3222,27.58,78,87,5.11,SO,1701111188
3,3,aykhal,66.0000,111.5000,-23.74,100,96,3.97,RU,1701111188
4,4,ushuaia,-54.8000,-68.3000,13.81,30,0,3.09,AR,1701111188



---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [93]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5,
    frame_width = 800,
    frame_height = 600,
    alpha = 0.75)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [95]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 29) &
                        (city_data_df["Max Temp"] > 20
                        )]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
41,41,mermoz boabab,14.7065,-17.4758,28.31,61,0,2.06,SN,1701111200
66,66,belmonte,-15.8631,-38.8828,27.70,58,0,3.85,BR,1701111208
136,136,sur,22.5667,59.5289,24.72,61,0,3.84,OM,1701111227
168,168,dangriga,16.9692,-88.2321,28.45,70,0,3.68,BZ,1701111236
177,177,hafar al-batin,28.4328,45.9708,20.32,56,0,3.09,SA,1701111238
198,198,seminole,28.7086,-81.2081,22.05,66,0,3.60,US,1701111195
203,203,ormara,25.2088,64.6357,24.46,52,0,0.72,PK,1701111246
217,217,isla vista,34.4133,-119.8610,22.95,29,0,2.06,US,1701111251
230,230,turbat,26.0023,63.0440,22.15,66,0,1.34,PK,1701111254
256,256,pisco,-13.7000,-76.2167,23.03,64,0,4.63,PE,1701111262



### Step 3: Create a new DataFrame called `hotel_df`.

In [96]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity})
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
41,mermoz boabab,SN,14.7065,-17.4758,61,
66,belmonte,BR,-15.8631,-38.8828,58,
136,sur,OM,22.5667,59.5289,61,
168,dangriga,BZ,16.9692,-88.2321,70,
177,hafar al-batin,SA,28.4328,45.9708,56,
198,seminole,US,28.7086,-81.2081,66,
203,ormara,PK,25.2088,64.6357,52,
217,isla vista,US,34.4133,-119.8610,29,
230,turbat,PK,26.0023,63.0440,66,
256,pisco,PE,-13.7000,-76.2167,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [97]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat, lon from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mermoz boabab - nearest hotel: No hotel found
belmonte - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
dangriga - nearest hotel: No hotel found
hafar al-batin - nearest hotel: No hotel found
seminole - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
isla vista - nearest hotel: No hotel found
turbat - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
chhor - nearest hotel: No hotel found
nasiriyah - nearest hotel: No hotel found
todos santos - nearest hotel: No hotel found
pacocha - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
aswan - nearest hotel: No hotel found
timbuktu - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
bandar-e lengeh - nearest hotel: Санаторый Спадарожнік
spanish wells - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
41,mermoz boabab,SN,14.7065,-17.4758,61,No hotel found
66,belmonte,BR,-15.8631,-38.8828,58,No hotel found
136,sur,OM,22.5667,59.5289,61,No hotel found
168,dangriga,BZ,16.9692,-88.2321,70,No hotel found
177,hafar al-batin,SA,28.4328,45.9708,56,No hotel found
198,seminole,US,28.7086,-81.2081,66,No hotel found
203,ormara,PK,25.2088,64.6357,52,No hotel found
217,isla vista,US,34.4133,-119.8610,29,No hotel found
230,turbat,PK,26.0023,63.0440,66,No hotel found
256,pisco,PE,-13.7000,-76.2167,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [98]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color="City",
    scale=0.5,
    frame_width = 800,
    frame_height = 600,
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)